In [107]:
import pickle
from copy import deepcopy
import math
import pandas as pd

activities_to_classify = {
        'Get/replace items from refrigerator/cabinets/drawers': 0,
        'Peel a cucumber': 1,
        'Clear cutting board': 2,
        'Slice a cucumber': 3,
        'Peel a potato': 4,
        'Slice a potato': 5,
        'Slice bread': 6,
        'Spread almond butter on a bread slice': 7,
        'Spread jelly on a bread slice': 8,
        'Open/close a jar of almond butter': 9,
        'Pour water from a pitcher into a glass': 10,
        'Clean a plate with a sponge': 11,
        'Clean a plate with a towel': 12,
        'Clean a pan with a sponge': 13,
        'Clean a pan with a towel': 14,
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 15,
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 16,
        'Stack on table: 3 each large/small plates, bowls': 17,
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 18,
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 19,
}

FPS = 30
ACTION_LENGTH = 5

def augmentation(data_frame):
    augmented_data = []

    for _, row in data_frame.iterrows():
        start_frame = row['start_frame']
        stop_frame = row['stop_frame']
        interval_size = FPS * ACTION_LENGTH 

        num_intervals = math.ceil((stop_frame - start_frame + 1) / interval_size)

        for i in range(num_intervals):
            new_start = start_frame + i * interval_size
            new_stop = min(new_start + interval_size - 1, stop_frame)  
            new_row = row.copy()
            new_row['start_frame'] = new_start
            new_row['stop_frame'] = new_stop
            new_row['start_timestamp'] = new_start/FPS
            new_row['stop_timestamp'] = new_stop/FPS
            augmented_data.append(new_row)

    augmented_dataframe = pd.DataFrame(augmented_data, columns=data_frame.columns)
    augmented_dataframe.reset_index(drop=True, inplace=True)

    return augmented_dataframe

def create_annotations_file(timestamp_file, annotations_file, type='train'):

    timestamps = pd.read_pickle(timestamp_file)
    timestamps = timestamps.drop(
        ['myo_left_timestamps', 'myo_right_timestamps', 'myo_left_readings', 'myo_right_readings'], axis=1)
    timestamps = timestamps.reset_index()

    start_timestamp = timestamps['start'].iloc[0]
    timestamps['start_frame'] = ((timestamps['start'] - start_timestamp) * FPS).astype(int)
    timestamps['stop_frame'] = ((timestamps['stop'] - start_timestamp) * FPS).astype(int)
    
    annotations = pd.read_pickle(annotations_file)
    annotations = annotations[annotations['file'] == 'S04_1.pkl']
    annotations = annotations.drop(['labels'], axis=1)
    
    complete_labels = pd.merge(timestamps, annotations, on='index', how='inner')
    
    complete_labels['uid'] = complete_labels['index']
    complete_labels['participant_id'] = 'S04'
    complete_labels['video_id'] = 'S04_1'
    complete_labels['verb'] = complete_labels['description_x']
    complete_labels['narration'] = complete_labels['description_x']
    complete_labels['verb_class'] = complete_labels['verb'].map(activities_to_classify)

    complete_labels = complete_labels[
        ['uid', 'participant_id', 'video_id', 'narration', 'start', 'stop', 'start_frame',
         'stop_frame', 'verb', 'verb_class']]
    
    complete_labels['type'] = type
    
    return complete_labels

def change_uid_to_emg(emg_data, split):
    emg_data = pd.read_pickle(emg_data)
    emg_data['uid'] = emg_data.reset_index().index + 1
    emg_data.to_pickle(f'new_emg_data_{split}.pkl')    


def take_S04_annotations_RGB(timestamps, emg_data, type):
    
    calibration_val = pd.read_pickle(timestamps)['start'].iloc[0]

    emg_data = pd.read_pickle(emg_data)
    
    emg_data = emg_data[emg_data['file'] == 'S04_1.pkl']
    
    emg_data = emg_data.rename(columns={'file': 'video_id', 'description': 'narration', 'description_class': 'verb_class'})
    emg_data['participant_id'] = 'S04'
    emg_data['video_id'] = 'S04_1'
    emg_data['start_frame'] = ((emg_data['start'] - calibration_val) * FPS).astype(int)
    emg_data['stop_frame'] = ((emg_data['stop'] - calibration_val) * FPS).astype(int)
    emg_data['verb'] = emg_data['narration']
    emg_data = emg_data.drop(['emg_data'], axis = 1)
    emg_data = emg_data[
        ['uid', 'participant_id', 'video_id', 'narration', 'start', 'stop', 'start_frame',
         'stop_frame', 'verb', 'verb_class']]
    
    emg_data.to_pickle(f'an_annotations_rgb/S04_{type}.pkl')    

def create_reduced_annotations(train_annotations, test_annotations):
    
    combined_df = pd.concat([train_annotations, test_annotations], ignore_index=True)
    
    combined_df = combined_df.sample(frac=1)

    combined_df.reset_index(inplace=True)
    combined_df['uid'] = combined_df.index
    
    train_df_final = combined_df[combined_df['type'] == 'train']
    test_df_final = combined_df[combined_df['type'] == 'test']
    
    train_df_final = train_df_final.drop(['type'], axis=1)
    test_df_final = test_df_final.drop(['type'], axis=1)

    train_df_final.to_pickle(f"an_annotations_rgb/S04_train.pkl")    
    test_df_final.to_pickle(f"an_annotations_rgb/S04_test.pkl")   
    
def create_multimodal_annotations(full_data, split, spectogram):
    
    full_data = pd.read_pickle(full_data)
    full_data = full_data[full_data['file'] == 'S04_1.pkl']
    full_data = full_data.rename(columns={'description_class': 'verb_class'})
    full_data['participant_id'] = 'S04'
    full_data['video_id'] = 'S04_1'
    
    create_emg_features(deepcopy(full_data), split, spectogram)
    
    final_annotations = full_data[
        ['uid', 'participant_id', 'video_id', 'description', 'verb_class']]
    
    final_annotations.to_pickle(f"an_multimodal_annotations/S04_{split}.pkl")   

def create_emg_features(full_data, split, spectogram):
    
    full_data = full_data[full_data['file'] == 'S04_1.pkl']
    full_data = full_data.rename(columns={'emg_data': 'features_EMG'})
    
    emg_features = full_data[
        ['uid', 'features_EMG']]
    emg_features = emg_features.to_dict('list')
    emg_features = {'features': emg_features}

    features_name = f'saved_features_an_multimodal/features_emg_spectogram_S04_{split}.pkl' if spectogram \
        else f'saved_features_an_multimodal/features_emg_S04_{split}.pkl'
      
    with open(features_name, 'wb') as f:
        pickle.dump(emg_features, f)
          
def create_emg_spec_features(full_data, split, spectrogram):
    
    full_data = pd.read_pickle(full_data)
    
    full_data = full_data.rename(columns={'spectrogram': 'features_EMG_spectrogram'})
    
    print(split)
    emg_features = full_data[
        ['uid', 'features_EMG_spectrogram']]
    emg_features= emg_features.to_dict(orient='index')
    emg_features = {'features': list(emg_features.values())}
    # Creazione del dizionario desiderato

    features_name = f'saved_features_an_multimodal/features_emg_80fs_spectrogram_allData_{split}.pkl' if spectrogram \
        else f'saved_features_an_multimodal/features_emg_80fs_allData_{split}.pkl'
      
    with open(features_name, 'wb') as f:
        pickle.dump(emg_features, f)

In [ ]:
take_S04_annotations_RGB(timestamps='action-net/S04_1.pkl', emg_data='EMG_data/emg/new_emg_data_test.pkl', type='test')
take_S04_annotations_RGB(timestamps='action-net/S04_1.pkl', emg_data='EMG_data/emg/new_emg_data_train.pkl', type='train')

In [ ]:
create_multimodal_annotations(full_data="new_emg_data_train.pkl", split='train', spectogram=False)
create_multimodal_annotations(full_data="new_emg_data_test.pkl", split='test', spectogram=False)

In [ ]:
create_emg_features(full_data="new_emg_data_train.pkl", split='train')
create_emg_features(full_data="new_emg_data_test.pkl", split='test')

In [108]:
create_emg_spec_features(full_data="C:/Users/Laura/Desktop/Universita/Polito/Advanced Machine Learning/aml23-ego/EMG_data/emg_spectrogram_80fs_test.pkl", spectrogram=True, split='test')
create_emg_spec_features(full_data="C:/Users/Laura/Desktop/Universita/Polito/Advanced Machine Learning/aml23-ego/EMG_data/emg_spectrogram_80fs_train.pkl", spectrogram=True, split='train')

test
train


In [ ]:
import pickle
import pandas as pd
with open('new_emg_data_top_test.pkl', 'rb') as f_pickle:
      dati=pickle.load(f_pickle)
      print(dati)
      df=pd.DataFrame(dati)
      
      # print(df.to_string(index=False, justify='right'))


In [ ]:
pd.read_pickle("C:/Users/Laura/Desktop/Universita/Polito/Advanced Machine Learning/aml23-ego/EMG_data/emg_spectrogram_80fs_test.pkl")

In [ ]:
import pandas as pd

data2 = pd.read_pickle('C:/Users/Laura/Desktop/Universita/Polito/Advanced Machine Learning/aml23-ego/saved_features_an_multimodal/features_emg_spectrogram_80fs_allData_test.pkl')

data2

In [110]:
import pandas as pd

df=pd.DataFrame(pd.read_pickle('C:/Users/Laura/Desktop/Universita/Polito/Advanced Machine Learning/aml23-ego/EMG_data/emg_spectrogram_80fs_test.pkl'))



In [111]:

print(df.columns)
duplicated_uids = df[df['uid'].duplicated(keep=False)]
# This will give you a DataFrame containing only rows with duplicate uids
print(duplicated_uids)

Index(['spectrogram', 'label', 'uid', 'file'], dtype='object')
Empty DataFrame
Columns: [spectrogram, label, uid, file]
Index: []
